In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import mahotas
import h5py
import imageio as iio
import os
from os import listdir
from itertools import chain
import pickle
from catboost import CatBoostClassifier
from sklearn import metrics

In [2]:
with open('X_train.pkl', 'rb') as f:
    X_train = pickle.load(f)

with open('y_train.pkl', 'rb') as f:
    y_train = pickle.load(f)

with open('X_test.pkl', 'rb') as f:
    X_test = pickle.load(f)

with open('y_test.pkl', 'rb') as f:
    y_test = pickle.load(f)

In [21]:
model = CatBoostClassifier(iterations=100,learning_rate = 0.9 , depth = 5 , eval_metric = 'Accuracy')

In [22]:
model.fit(X_train,y_train,eval_set=(X_test , y_test))

0:	learn: 0.2550091	test: 0.2095588	best: 0.2095588 (0)	total: 629ms	remaining: 1m 2s
1:	learn: 0.2877960	test: 0.2573529	best: 0.2573529 (1)	total: 1.32s	remaining: 1m 4s
2:	learn: 0.3214936	test: 0.2536765	best: 0.2573529 (1)	total: 1.81s	remaining: 58.5s
3:	learn: 0.3561020	test: 0.2867647	best: 0.2867647 (3)	total: 2.22s	remaining: 53.2s
4:	learn: 0.3761384	test: 0.3088235	best: 0.3088235 (4)	total: 2.65s	remaining: 50.4s
5:	learn: 0.4025501	test: 0.2941176	best: 0.3088235 (4)	total: 3.06s	remaining: 48s
6:	learn: 0.4398907	test: 0.3235294	best: 0.3235294 (6)	total: 3.52s	remaining: 46.8s
7:	learn: 0.4526412	test: 0.3272059	best: 0.3272059 (7)	total: 3.99s	remaining: 45.9s
8:	learn: 0.4562842	test: 0.3088235	best: 0.3272059 (7)	total: 4.4s	remaining: 44.5s
9:	learn: 0.4854281	test: 0.3088235	best: 0.3272059 (7)	total: 4.81s	remaining: 43.3s
10:	learn: 0.5054645	test: 0.3235294	best: 0.3272059 (7)	total: 5.22s	remaining: 42.3s
11:	learn: 0.5336976	test: 0.3345588	best: 0.3345588 (11

94:	learn: 0.9963570	test: 0.4080882	best: 0.4411765 (73)	total: 42.2s	remaining: 2.22s
95:	learn: 0.9963570	test: 0.4080882	best: 0.4411765 (73)	total: 42.6s	remaining: 1.78s
96:	learn: 0.9972678	test: 0.4117647	best: 0.4411765 (73)	total: 43.1s	remaining: 1.33s
97:	learn: 0.9972678	test: 0.4044118	best: 0.4411765 (73)	total: 43.5s	remaining: 888ms
98:	learn: 0.9972678	test: 0.4044118	best: 0.4411765 (73)	total: 43.9s	remaining: 443ms
99:	learn: 0.9972678	test: 0.4117647	best: 0.4411765 (73)	total: 44.3s	remaining: 0us

bestTest = 0.4411764706
bestIteration = 73

Shrink model to first 74 iterations.


In [ ]:
y_pred = model.predict(X_test)

In [15]:
print(metrics.classification_report(y_pred = y_pred , y_true = y_test))

              precision    recall  f1-score   support

           0       0.50      0.46      0.48        26
           1       0.37      0.39      0.38        28
           2       0.36      0.37      0.36        27
           3       0.43      0.60      0.50        30
           4       0.33      0.27      0.30        26
           5       0.35      0.29      0.31        28
           6       0.56      0.54      0.55        26
           7       0.44      0.43      0.44        28
           8       0.52      0.59      0.55        27
           9       0.52      0.42      0.47        26

    accuracy                           0.44       272
   macro avg       0.44      0.44      0.43       272
weighted avg       0.44      0.44      0.43       272



In [59]:
from sklearn import svm
clf = svm.SVC(C=1 , kernel = 'linear')

In [60]:
clf.fit(X_train, y_train)

SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [61]:
y_pred = clf.predict(X_test)
print(metrics.classification_report(y_pred = y_pred , y_true = y_test))

              precision    recall  f1-score   support

           0       0.42      0.42      0.42        26
           1       0.39      0.54      0.45        28
           2       0.26      0.30      0.28        27
           3       0.42      0.43      0.43        30
           4       0.35      0.27      0.30        26
           5       0.17      0.14      0.15        28
           6       0.36      0.31      0.33        26
           7       0.29      0.25      0.27        28
           8       0.42      0.59      0.49        27
           9       0.44      0.31      0.36        26

    accuracy                           0.36       272
   macro avg       0.35      0.36      0.35       272
weighted avg       0.35      0.36      0.35       272



In [56]:
from xgboost import XGBClassifier
xgb =  XGBClassifier(eta = 0.01 , gamma =10 , max_depth =12)

In [57]:
xgb.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False, eta=0.01,
              eval_metric=None, gamma=10, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.00999999978, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=12, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, objective='multi:softprob',
              predictor='auto', random_state=0, ...)

In [37]:
y_pred = xgb.predict(X_test)
print(metrics.classification_report(y_pred = y_pred , y_true = y_test))

              precision    recall  f1-score   support

           0       0.48      0.46      0.47        26
           1       0.41      0.43      0.42        28
           2       0.34      0.41      0.37        27
           3       0.40      0.47      0.43        30
           4       0.23      0.19      0.21        26
           5       0.30      0.25      0.27        28
           6       0.56      0.58      0.57        26
           7       0.39      0.39      0.39        28
           8       0.48      0.56      0.52        27
           9       0.50      0.38      0.43        26

    accuracy                           0.41       272
   macro avg       0.41      0.41      0.41       272
weighted avg       0.41      0.41      0.41       272



In [69]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=2000, random_state=0)

In [70]:
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=2000,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [71]:
y_pred = rf.predict(X_test)
print(metrics.classification_report(y_pred = y_pred , y_true = y_test))

              precision    recall  f1-score   support

           0       0.47      0.58      0.52        26
           1       0.37      0.46      0.41        28
           2       0.75      0.33      0.46        27
           3       0.37      0.43      0.40        30
           4       0.31      0.19      0.24        26
           5       0.35      0.29      0.31        28
           6       0.49      0.65      0.56        26
           7       0.36      0.36      0.36        28
           8       0.55      0.63      0.59        27
           9       0.52      0.50      0.51        26

    accuracy                           0.44       272
   macro avg       0.45      0.44      0.44       272
weighted avg       0.45      0.44      0.43       272

